# Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание *
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"


In [1]:
import requests

# Выгрузка данных по строящимся объектам

In [2]:
# Получение кратких данных по всем объектам
# Объектов на момент создания программы 10075
offset_ = 0
limit_ = 1000 # За раз 1000 отрабатывает стабильно
iteration = 11 # Чтобы обработать 10075 объектов нужно 11 итераций (11*1000>10075)
objects_list = []
for i in range(0, iteration):
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset=' \
          f'{offset_}&limit=' \
          f'{limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    res = requests.get(url)
    objects_data = res.json()
    objects_list.extend(objects_data.get('data').get('list'))
    offset_ += limit_

In [3]:
# Получиние ID объектов
objids = [x.get('objId') for x in objects_list]
len(objids)

10075

In [4]:
# Получение расширенных данных по каждому объекту
# objids = [13383, 13397, 13398] # Для теста
# Для обработки всех объектов нужно ~30 минут
objects_list_full = []
i = 0
for objid in objids:

    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/' \
          f'{objid}'
    res_full = requests.get(url)
    objects_data_full = res_full.json()
    objects_list_full.append(objects_data_full.get('data'))
    i += 1
    print(i, '\r', end='') # Для видимости прогресса обработки в консоли

# Cохранение данных в pandas dataframe

In [5]:
import pandas as pd

In [6]:
df = pd.json_normalize(objects_list_full)

# Cохранение pandas dataframe в excel

In [7]:
df.to_excel('dmrf.xlsx', index = False)

# Cохранение pandas dataframe в pkl

In [8]:
df.to_pickle('dmrf.pkl')

# Cохранение pandas dataframe в БД

In [ ]:
from sqlalchemy import create_engine

# Подключение к БД
user = "root" # Ввод имени пользователя в кавычках
password = "root" # Ввод пароля в кавычках
dsn = "localhost:5432/dmrf" # Ввод IP сервера БД, порта и имени БД
engine = create_engine(f'postgresql://{user}:{password}@{dsn}')

df.to_sql(name='dmrf_tbl',
            con=engine,
            if_exists='replace',
            index=False)

# Скачивание фото объектов

In [34]:
import os

# objids = [13383, 13397, 13398] # Для теста
for objid in objids:

    # Получение photoRenderDTO объекта
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/' \
          f'{objid}'
    res = requests.get(url)
    objects_data = res.json()
    photoRenderDTO = objects_data.get('data').get('photoRenderDTO')

    i = 1
    # Перебор всех objRenderPhotoUrl для объекта
    for photo in photoRenderDTO:
        url_photo = photo.get('objRenderPhotoUrl')
        res_photo = requests.get(url_photo)

        # Создание директории для сохранения фото
        if not os.path.isdir(f'foto\\{objid}'):
            os.mkdir(f'foto\\{objid}')

        # Создание файла и сохранение в него фото
        with open(f'foto\\{objid}\\{objid}_{i}.jpg', 'wb') as f:
            f.write(res_photo.content)

        i += 1